## Imports

In [12]:
from os.path import join
import numpy as np
from osgeo import gdal, osr
from glob import glob
from datetime import date

## Filepaths and parameter settings

In [16]:
# filepaths
datadir = '/Volumes/ASO_JPL/pinatubo_geoeng_data'
indir = join(datadir,'raw')
intdir = join(datadir,'ee_tiffs')
outdir = join('..','..','results')

# identify variables to pull from daily and monthly output
var_mon = ['clt','pr','od550aer','od870aer']
var_day = ['clt','tas','tasmin','tasmax']

In [33]:
## select filenames to append together before exporting as GTiff


# First, select date range
start = date(2019,12,1)
end = date(2089,12,30)


# then sift through files
fnames = {'rcp45':{'mon':{},'day':{}},
          'G3':{'mon':{},'day':{}}}

for proj in ['rcp45','G3']:
    fn = fnames[proj]
    
    for var in var_mon:
        files = glob(join(indir,var+'_Amon_HadGEM2-ES_'+proj+'*'))
        starts = [date(int(f[-16:-12]),int(f[-12:-10]),1) for f in files]
        ends = [date(int(f[-9:-5]),int(f[-5:-3]),30) for f in files]

        for i, 
    

[datetime.date(2030, 11, 30),
 datetime.date(2055, 11, 30),
 datetime.date(2080, 11, 30),
 datetime.date(2099, 11, 30),
 datetime.date(2124, 11, 30),
 datetime.date(2149, 11, 30),
 datetime.date(2174, 11, 30),
 datetime.date(2199, 11, 30),
 datetime.date(2224, 11, 30),
 datetime.date(2249, 11, 30),
 datetime.date(2274, 11, 30),
 datetime.date(2299, 11, 30),
 datetime.date(2299, 12, 30),
 datetime.date(2030, 11, 30),
 datetime.date(2055, 11, 30),
 datetime.date(2080, 11, 30),
 datetime.date(2100, 11, 30),
 datetime.date(2100, 12, 30),
 datetime.date(2030, 11, 30),
 datetime.date(2055, 11, 30),
 datetime.date(2080, 11, 30),
 datetime.date(2100, 11, 30),
 datetime.date(2100, 12, 30)]

In [11]:
fnames['G3']['day']['clt']

['/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20191201-20291130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20291201-20391130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20391201-20491130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20491201-20591130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20591201-20691130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20691201-20791130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20791201-20891130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r1i1p1_20891201-20891230.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r2i1p1_20191201-20291130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/raw/clt_day_HadGEM2-ES_G3_r2i1p1_20291201-20391130.nc',
 '/Volumes/ASO_JPL/pinatubo_geoeng_data/

In [90]:
ds_in = gdal.Open(join(indir,'tasmin_Amon_HadGEM2-ES_rcp45_r1i1p1_208012-209910.nc'))
proj_out = osr.SpatialReference()
proj_out.ImportFromEPSG(4326)
driver = gdal.GetDriverByName( 'GTiff' )

gt_in = ds_in.GetGeoTransform()
gt_out = [gt_in[0]-180]
for i in range(1,len(gt)):
    gt_out.append(gt_in[i])
gt_out = tuple(gt_out)

md_all = ds_in.GetMetadata()

In [91]:
subdatasets = ds_in.GetSubDatasets()
sds = {}
for subdataset in subdatasets:

    sds[subdataset[1].split(" ")[1]] = gdal.Open(subdataset[0])

longs = sds['lon_bnds'].GetRasterBand(1).ReadAsArray()
lats = sds['lat_bnds'].GetRasterBand(1).ReadAsArray()

vals = []
n_bnds = sds['air_temperature'].RasterCount

# get size of input NetCDFs
x_size = sds['air_temperature'].GetRasterBand(1).XSize
y_size = sds['air_temperature'].GetRasterBand(1).YSize

# set output tiff to have one more longitude column
ds_out = driver.Create(join(intdir,'out.tiff'), x_size+1, y_size, n_bnds, gdal.GDT_Float32 )
ds_out.SetProjection(proj_out.ExportToWkt())
ds_out.SetGeoTransform(gt_out)
ds_out.SetMetadata(md_all)

half_x = x_size/2
for i in range(n_bnds):
    src_band = sds['air_temperature'].GetRasterBand(i+1)
    east_hemi_data = src_band.ReadRaster(0, 0, half_x, src_band.YSize)
    west_hemi_data = src_band.ReadRaster(half_x, 0, half_x, src_band.YSize)
    east_column = src_band.ReadRaster(half_x, 0, 1, src_band.YSize)
    md = src_band.GetMetadata()
    
    dst_band = ds_out.GetRasterBand(i+1)
    dst_band.WriteRaster(0, 0, half_x, src_band.YSize, west_hemi_data)
    dst_band.WriteRaster(half_x, 0, half_x, src_band.YSize, east_hemi_data)
    dst_band.WriteRaster(half_x*2, 0, 1, src_band.YSize, east_column)
    dst_band.SetMetadata(md)

sds = None
ds_out = None

# BELOW THIS IS OLD, TO BE DELETED

## Functions

In [10]:
def copy_dims(dsin,dsout,extra_lon=False):
    for dname, the_dim in dsin.dimensions.iteritems():
        if extra_lon and dname == 'lon': len_dim = len(the_dim) + 1
        else: len_dim = len(the_dim)
        dsout.createDimension(dname, len_dim if not the_dim.isunlimited() else None)
    return dsout

def copy_vars(dsin,dsout,extra_lon=False):
    # Copy variables aside from variable of interest
    for v_name in ['time','lat','lon','lon_bnds','time_bnds','lat_bnds','od870aer']:
        varin = dsin.variables[v_name]
        outVar = dsout.createVariable(v_name, varin.dtype, varin.dimensions)

        print v_name
        # Copy variable attributes
        outVar.setncatts({k: getattr(varin,k) for k in varin.ncattrs()})

        # Copy variable values
        if extra_lon==True and v_name in ['lon','lon_bnds','od870aer']:
            if v_name=='lon':
                outVar[:-1] = varin[:]
            if v_name=='lon_bnds':
                outVar[:-1,:] = varin[:]
            if v_name=='od870aer':
                outVar[:,:,:-1] = varin[:]
        else:
            outVar[:] = varin[:]
        
    return dsout

## Prepare processed data NetCDF file

In [52]:
dsout.close()

In [53]:
## Merge files for one variable, one ensemble to get dimensions

# use SAOD870 as one example of a monthly observation to pull dimensions from
dsin = nc.Dataset(join(indir,'od870aer_aero_HadGEM2-ES_rcp45_r1i1p1_217412-219911.nc'))

# output to a single file for CMIP5 reference scenario results
dsout = nc.Dataset(int_data_rcp45,'w',format='NETCDF4')

# copy over dimensions
dsout = copy_dims(dsin, dsout,extra_lon=True)

# add ensemble member dimension
# ens_dim = dsout.createDimension('ensemble',4)
# ens = dsout.createVariable('ensemble','u1',('ensemble',))
# ens[:] = [1,2,3,4]

# copy over dimension-based variables
dsout = copy_vars(dsin, dsout,extra_lon=True)


time
lat
lon
lon_bnds
time_bnds
lat_bnds
od870aer


In [54]:
dsout.variables['lon'][:-1] = np.concatenate((dsout.variables['lon'][96:-1]-360,dsout.variables['lon'][:96]))
dsout.variables['od870aer'][:,:,:-1] = np.concatenate((dsout.variables['od870aer'][:,:,96:-1],dsout.variables['od870aer'][:,:,:96]),axis=2)
dsout.variables['lon_bnds'][:-1,:] = np.concatenate((dsout.variables['lon_bnds'][96:-1,:]-360,dsout.variables['lon_bnds'][:96,:]),axis=0)

In [55]:
dsout.variables['lon'][-1] = dsout.variables['lon'][0]+360

dsout.variables['lon_bnds'][-1,:] = dsout.variables['lon_bnds'][0,:]+360

dsout.variables['od870aer'][:,:,-1] = dsout.variables['od870aer'][:,:,0]

In [56]:
dsout.close()

## Pull CMIP5 (RCP 4.5) data

In [190]:
ensemble_members = [1]

for e in ensemble_members:
    
    ## cloud fraction, SAOD 870, and 560
#     for var in ['od870aer_aero','od560aer_aero','clt_atmos']:
    for var in ['od870aer_aero']:
        if '_aero' in var:
            varname = var[:-5]
        elif '_atmos' in var:
            varname = var[:-6]
            
        dsin = nc.MFDataset(join(indir,'{}_HadGEM2-ES_rcp45_r{}i1p1_*.nc'.format(var,e)))

        # Add value of interest
        with nc.Dataset(int_data_rcp45,'a',format='NETCDF4') as dsout:
            newvar = dsout.createVariable(varname,'f8',('time','lat','lon','ensemble',))
            newvar[:,:,:,e-1] = dsin.variables[varname][:]
            
    ## TODO: do this for temp but bin by days in a month that fall into a given category
    
    

## Pull GeoMIP (G3) data

## Get betas

## Calculate effects (CMIP5 scenario, GeoMIP scenario, and difference) for each variable and then the cumulative effect

1) first calculate for each ensemble individually
2) Then take mean